In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [8]:
tool.name

'wikipedia'

In [13]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(text_splitter, OllamaEmbeddings(model="llama2:latest"))
retriever = vectordb.as_retriever()
retriever

In [ ]:
from langchain.tools.retriever import create_retriever_tool

create_retriever_tool(retriever, "langsmith_search",
                    "Search for information about Langsmith.")

'wikipedia'